# Load Data

In [1]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import mnet

Create path to access local files

In [2]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()

/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: local metadata cache: starting processing of paths: 1
/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1589236836.2014117
bundles.py: WARNING: Skipped 1 sources already in cache.
/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: writing time: 0.019 md items: 6

/opt/anaconda3/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
caching.py: WARNING: cache: final writing time: 0.019 seconds


Save Telemann Fantasie No. 1 as a stream object

In [3]:
s = music21.corpus.parse('telemannfantasie1.xml')

Inspect stream to see where desired parts are located

Save Solo Flute part

In [4]:
flute = s[5]

# Discreet Sections Pipeline

Define groups as list where each entry is the measure that a new section starts. Need the last entry to be "end"

In [5]:
flute_notes =flute.recurse().notes

In [6]:
grouping = [1, 5, 11, 27, 37, 49, 61, 75, "end"]

Use the convert_grouping function to convert note list and groups into a list of nodes. Will also return list of section-transition edges

In [7]:
nodelst_group, transition_edges=mnet.convert_grouping(flute_notes, grouping)


Create Graph

In [8]:
g_group=mnet.create_graph(nodelst_group)
#Write to .gexf
#nx.write_gexf(g, "grouped_composition.gexf")

Generate Random Walk

In [9]:
randomwalk_group=mnet.generate_randomwalk(g_group)

Convert random walk to back to music.

Uses conversion function group_strto16thnote, which takes a group pitch string and converts it to a note with length of 16th note. Notes preceeding transition edges will be half notes

In [10]:
def convert_to_stream(notelist):

    s = music21.stream.Stream()
    for thisNote in notelist:
        s.append(thisNote)
    return s


In [11]:
tune = mnet.group_strto16thnote(randomwalk_group)
new_composition_group = convert_to_stream(tune)
#Write to MIDI
new_composition_group.write('xml', "../xml/Forced_groups.xml")

'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/Forced_groups.xml'

## Alter Weight of Transition Nodes

Currently all transition edges have a weight of 1. This can be altered through degree_increase()

In [13]:
graph_adjusted1 = mnet.degree_increase(g_group, transition_edges, 2)

In [14]:
randomwalk_group_adj=mnet.generate_randomwalk(graph_adjusted1)
tune = mnet.group_strto16thnote(randomwalk_group_adj)
new_composition_group = convert_to_stream(tune)
#Write to MIDI
new_composition_group.write('xml', "../xml/Forced_groups_with_edge_alteration.xml")

'/Users/kaitlinpet/Desktop/Coursework/networks/project/xml/Forced_groups_with_edge_alteration.xml'